In [2]:
!pip install -q --upgrade ipython==5.5.0
!pip install -q --upgrade ipykernel==4.6.0

     |████████████████████████████████| 768kB 1.4MB/s 
     |████████████████████████████████| 245kB 57.1MB/s 
ERROR: google-colab 1.0.0 has requirement ipykernel~=4.6.0, but you'll have ipykernel 5.1.3 which is incompatible.
     |████████████████████████████████| 112kB 1.4MB/s 


In [0]:
from google.colab import files
files.upload()

In [0]:
!pip install -q kaggle

In [0]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

In [0]:
!chmod 600 ~/.kaggle/kaggle.json

In [0]:
!kaggle competitions download -c dogs-vs-cats #had to accept the term and conditions of the competition

In [0]:
!unzip train.zip
!unzip test1.zip

In [3]:
import os, cv2, random
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from tqdm import tqdm
from random import shuffle 
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
from keras.utils import plot_model
from keras.applications import ResNet50
from keras.models import Sequential
from keras.layers import Dense, Flatten, GlobalAveragePooling2D
%matplotlib inline 

Using TensorFlow backend.


## Process data

In [9]:
!ls train | head -3

cat.0.jpg
cat.10000.jpg
cat.10001.jpg


In [10]:
!ls train | tail -3

dog.999.jpg
dog.99.jpg
dog.9.jpg


In [11]:
!ls test1 | head -3

10000.jpg
10001.jpg
10002.jpg


In [0]:
train = []

for file in os.listdir('train'):
  ## ------------------set label--------------------
  label = file.split('.')[0]
  if label=='cat':
    L = [0,1]
  elif label=='dog':
    L = [1,0]
  else:
    print("Error! Filename is",file)
    break
  
  ## ----------------set images----------------------
  img = cv2.imread('train/'+file,cv2.IMREAD_COLOR)
  img = cv2.resize(img, (224,224))
  img=img/255.0
  train.append((img,L))

In [0]:
shuffle(train)

In [0]:
len(train)

In [0]:
[x[1] for x in train[:10]]

In [0]:
test = []

for file in os.listdir('test1'):
  ## ------------------set label--------------------
  L = [-1,-1]
  
  ## ----------------set images----------------------
  img = cv2.imread('test1/'+file,cv2.IMREAD_COLOR)
  img = cv2.resize(img, (224,224))
  img=img/255.0
  test.append((img,L))

In [0]:
len(test)

## See Data

In [0]:
def show_images(data, isTest=False):
    f, ax = plt.subplots(5,5, figsize=(15,15))
    for i,data in enumerate(data[:25]):
        img_num = data[1]
        img_data = data[0]
        label = np.argmax(img_num)
        if label  == 0: 
            str_label='Dog'
        elif label == 1: 
            str_label='Cat'
        if(isTest):
            str_label="None"
        ax[i//5, i%5].imshow(img_data)
        ax[i//5, i%5].axis('off')
        ax[i//5, i%5].set_title("Label: {}".format(str_label))
    plt.show()

In [0]:
show_images(train)

In [0]:
show_images(test,isTest=True)

## Model

In [0]:
X = np.array([i[0] for i in train]).reshape(-1,224,224,3)
y = np.array([i[1] for i in train])

https://keras.io/applications/#classify-imagenet-classes-with-resnet50

https://keras.io/applications/#resnet (parameter details)

In [0]:
model = Sequential()
model.add(ResNet50(include_top=False, pooling='max'))
model.add(Dense(2, activation='softmax'))

In [0]:
# ResNet-50 model is already trained, should not be trained
model.layers[0].trainable = False

In [0]:
model.summary()

In [0]:
model.compile(optimizer='sgd', loss='categorical_crossentropy', metrics=['accuracy'])

In [0]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.3)

In [0]:
train_model = model.fit(X_train, y_train,
                  batch_size=64,
                  epochs=10,
                  verbose=1,
                  validation_data=(X_val, y_val))

## Evaluate

In [0]:
def plot_accuracy_and_loss(train_model):
    hist = train_model.history
    acc = hist['acc']
    val_acc = hist['val_acc']
    loss = hist['loss']
    val_loss = hist['val_loss']
    epochs = range(len(acc))
    f, ax = plt.subplots(1,2, figsize=(14,6))
    ax[0].plot(epochs, acc, 'g', label='Training accuracy')
    ax[0].plot(epochs, val_acc, 'r', label='Validation accuracy')
    ax[0].set_title('Training and validation accuracy')
    ax[0].legend()
    ax[1].plot(epochs, loss, 'g', label='Training loss')
    ax[1].plot(epochs, val_loss, 'r', label='Validation loss')
    ax[1].set_title('Training and validation loss')
    ax[1].legend()
    plt.show()

In [0]:
plot_accuracy_and_loss(train_model)

In [0]:
score = model.evaluate(X_val, y_val, verbose=0)
print('Validation loss:', score[0])
print('Validation accuracy:', score[1])

In [0]:
predicted_classes = model.predict_classes(X_val)
y_true = np.argmax(y_val,axis=1)

In [0]:
target_names = ["Class {}:".format(i) for i in range(2)]
print(classification_report(y_true, predicted_classes, target_names=target_names))

## Test set

In [0]:
x = test[0][0].reshape(-1,224,224,3)
out = model.predict([x])[0]
out

In [0]:
f, ax = plt.subplots(5,5, figsize=(15,15))
for i,data in enumerate(test[:25]):
    img_num = data[1]
    img_data = data[0]
    orig = img_data
    data = img_data.reshape(-1,224,224,3)
    model_out = model.predict([data])[0]
    
    if np.argmax(model_out) == 0: 
        str_predicted='Dog'
    else: 
        str_predicted='Cat'
    ax[i//5, i%5].imshow(orig)
    ax[i//5, i%5].axis('off')
    ax[i//5, i%5].set_title("Predicted:{}".format(str_predicted))    
plt.show()